# Teste com outro Dataset (Coleta 3)

## Objetivo
Recuperar o modelo treinado no notebook **05-treinamento_classificacao.ipynb** e aplicar o modelo já treinado a um novo conjunto de dados coletados num outro dia para verificar a acurácia.

O dataset considerado neste teste usa outras coletas realizadas entre **10:30 do dia 16/11/2022 e 10:15 do dia 17/11/2022**, correspondente a um período de 24 horas. As regras de coletas são as mesmas, coleta a cada 15 minutos durante 24horas.

### Métricas Consideradas:
- hash (identificador do contêiner)
- Consumo de CPU (Valores de pico e tendência central)
- Consumo de Memória (Valores de pico e tendência central)
- Flavor (rótulo)

In [1]:
import pandas as pd
import hashlib
import glob
import os
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

### Funções Gerais

In [2]:
# Extrai a identificação do sistema a partir do nome do ambiente
def extract_system_name(namespace):
    environment = namespace.split('-')[-1]
    system = namespace.split(f'-{environment}')[0]
    return system

# Cria os diretórios
def create_folder(path):
    try:
       os.makedirs(path)
    except FileExistsError:
       # directory already exists
       pass

def hash(ambiente, pod):
    return hashlib.md5(f'{ambiente}{pod}'.encode('utf-8')).hexdigest()

# Regra de Rotulação
def rotular_V2(cpu_MAX, cpu_MED, cpu_threshod, memoria_MAX, memoria_MED, memoria_threshod):
    for f in range(0,11):
        resultado_memoria = validar_flavor_para_memoria(memoria_MAX, memoria_MED, memoria_threshod, flavors[f])
        resultado_cpu = validar_flavor_para_cpu(cpu_MAX, cpu_MED, cpu_threshod, flavors[f])
        if resultado_memoria and resultado_cpu:
            return flavors[f]['flavor']
    return 'Nao classificado'

def validar_flavor_para_memoria(memoria_MAX, memoria_MED, memoria_threshod, flavor):
    return memoria_MAX + (memoria_threshod * 2 * (memoria_MAX - memoria_MED)) <= flavor['memoria']

def validar_flavor_para_cpu(cpu_MAX, cpu_MED,cpu_threshod, flavor):
    if cpu_MAX <= flavor['cpu']:
        return True
    return cpu_MAX - flavor['cpu'] <= cpu_threshod * 2 * (cpu_MAX - cpu_MED)

# Predict
def predict(model, exemplar):
    del exemplar['hash']
    del exemplar['flavor']    
    print(f'Predicao para os parâmetros [{exemplar}] => flavor {model.predict(exemplar)[0]}')

# Function to calculate accuracy
def cal_accuracy(y_test, y_pred):      
    print("Confusion Matrix: ",
        confusion_matrix(y_test, y_pred))
      
    print ("Accuracy : ",
    accuracy_score(y_test,y_pred)*100)
      
    print("Report : ",
    classification_report(y_test, y_pred))

### Pré-processamento dos arquivos coletados (cpu, memoria, error, throttled)
Pré-processamento dos arquivos originais coletados.

In [3]:
# Processamento de arquivos de CPU
header_list = ["Sistema", "Ambiente", "Modulo", "Pod", "Uso_CPU"]
path_processados = '/dados/coletas_dia_3/cpu/processados'
path_projeto = '/home/56740050368/Treinamento/IA-PUC_Minas/Trabalho_Cientifico/trabalho_puc_minas_2022'
path_metricas = '/dados/coletas_dia_3/cpu/'
csv_files = glob.glob(path_projeto+path_metricas+ "*.gz")

# Cria diretorios
create_folder(path_projeto+path_metricas)
create_folder(path_projeto+path_processados)

for file in csv_files:    
    data = pd.read_csv(file, sep=';', header=None, names=header_list)
    # obtem horario
    filename = file.replace(path_projeto+path_metricas, "")
    hora = int(filename[4:6])
    min = int(filename[6:8])
        
    # acrescenta coluna de horário    
    data['Hora'] = hora
    
    # acrescenta coluna de minuto    
    data['Minuto'] = min
    
    # acrescenta coluna de hash
    data['Hash'] = [hash(x, y) for x, y in zip(data['Ambiente'], data['Pod'])]
    
    # salva arquivo
    data.to_csv(f'{path_projeto}{path_processados}/cpu_{hora}_{min}.csv', index=False)
    
print('Fim processamento de CPU')

Fim processamento de CPU


In [4]:
# Processamento de dados de memória
header_list = ["Sistema", "Ambiente", "Modulo", "Pod", "Uso_Memoria"]
path_processados = '/dados/coletas_dia_3/memoria/processados'
path_metricas = '/dados/coletas_dia_3/memoria/'
csv_files = glob.glob(path_projeto+path_metricas+ "*.gz")

# Cria diretorios
create_folder(path_projeto+path_metricas)
create_folder(path_projeto+path_processados)

for file in csv_files:    
    data = pd.read_csv(file, sep=';', header=None, names=header_list)
    # obtem horario
    filename = file.replace(path_projeto+path_metricas, "")
    hora = int(filename[7:9])
    min = int(filename[9:11])
        
    # acrescenta coluna de horário    
    data['Hora'] = hora
    data['Minuto'] = min
    # acrescenta coluna de hash
    data['Hash'] = [hash(x, y) for x, y in zip(data['Ambiente'], data['Pod'])]
    
    # salva arquivo
    data.to_csv(f'{path_projeto}{path_processados}/memoria_{hora}_{min}.csv', index=False)
    
print('Fim processsamento de Memoria')

Fim processsamento de Memoria


In [5]:
# Processamento da coleta de pods encerrados por estouro de memória
header_list = ["Sistema", "Ambiente", "Modulo", "Pod", "Error", "Qtd"]
path_processados = '/dados/coletas_dia_3/error/processados'
path_metricas = '/dados/coletas_dia_3/error/'
csv_files = glob.glob(path_projeto+path_metricas+ "*.gz")

# Cria diretorios
create_folder(path_projeto+path_metricas)
create_folder(path_projeto+path_processados)

for file in csv_files:    
    data = pd.read_csv(file, sep=';', header=None, names=header_list)
    # obtem horario
    filename = file.replace(path_projeto+path_metricas, "")
    
    hora = int(filename[6:8])
    min = int(filename[8:10])   
        
    # acrescenta coluna de horário    
    data['Hora'] = hora
    data['Minuto'] = min
    # acrescenta coluna de hash
    data['Hash'] = [hash(x, y) for x, y in zip(data['Ambiente'], data['Pod'])]
    
    # salva arquivo
    data.to_csv(f'{path_projeto}{path_processados}/error_{hora}_{min}.csv', index=False)
    
print('Fim processsamento de Erros de Memória')

Fim processsamento de Erros de Memória


In [6]:
# Processamento da coleta de pods encerrados por estouro de memória
header_list = ["Sistema", "Ambiente", "Modulo", "Pod", "Uso_CPU"]
path_processados = '/dados/coletas_dia_3/throttled/processados'
path_metricas = '/dados/coletas_dia_3/throttled/'
csv_files = glob.glob(path_projeto+path_metricas+ "*.gz")

# Cria diretorios
create_folder(path_projeto+path_metricas)
create_folder(path_projeto+path_processados)

for file in csv_files:    
    data = pd.read_csv(file, sep=';', header=None, names=header_list)
    # obtem horario
    filename = file.replace(path_projeto+path_metricas, "")
    hora = int(filename[14:16])
    min = int(filename[16:18])   
        
    # acrescenta coluna de horário    
    data['Hora'] = hora
    data['Minuto'] = min
    # acrescenta coluna de hash
    data['Hash'] = [hash(x, y) for x, y in zip(data['Ambiente'], data['Pod'])]
    
    # salva arquivo
    data.to_csv(f'{path_projeto}{path_processados}/cpu_throttled_{hora}_{min}.csv', index=False)
    
print('Fim processsamento de CPU Throttled')

Fim processsamento de CPU Throttled


## Consolidação das Métricas num único arquivo por métrica

In [7]:
# Recuperar metricas dos arquivos processados
path_processados = '/dados/coletas_dia_3/cpu/processados'
csv_files = glob.glob(path_projeto+path_processados + "/*.csv")

df_list = (pd.read_csv(file, skiprows = 1,header = None) for file in csv_files)

# Concatenate all DataFrames
big_cpu_df  = pd.concat(df_list, ignore_index=True)

# Cria Tb Aplicações
tb_medidas_cpu = big_cpu_df[[7, 5, 6, 4]].copy() # Colunas hash, hora, minuto, consumo_cpu

# salva arquivo
tb_medidas_cpu.to_csv(f'{path_projeto}/{path_processados}/consolidado_cpu.csv', index=False, header=['hash', 'hora', 'min', 'consumo_cpu']) # 
    
print('Gerado arquivo consolidado da métrica de consumo de CPU')

Gerado arquivo consolidado da métrica de consumo de CPU


In [8]:
# Recuperar metricas dos arquivos processados
path_processados = '/dados/coletas_dia_3/memoria/processados'
csv_files = glob.glob(path_projeto+path_processados + "/*.csv")

df_list = (pd.read_csv(file, skiprows = 1,header = None) for file in csv_files)

# Concatenate all DataFrames
big_memoria_df  = pd.concat(df_list, ignore_index=True)

# Cria Tb Aplicações
tb_medidas_memoria = big_memoria_df[[7, 5, 6, 4]].copy() # Colunas hash, hora, minuto, consumo_memoria

# salva arquivo
tb_medidas_memoria.to_csv(f'{path_projeto}/{path_processados}/consolidado_memoria.csv', index=False, header=['hash', 'hora', 'min', 'consumo_memoria']) 
    
print('Gerado arquivo consolidado da métrica de consumo de Memória')

Gerado arquivo consolidado da métrica de consumo de Memória


In [9]:
# Recuperar metricas dos arquivos processados
path_processados = '/dados/coletas_dia_3/error/processados'
csv_files = glob.glob(path_projeto+path_processados + "/*.csv")

df_list = (pd.read_csv(file, skiprows = 1,header = None) for file in csv_files)

# Concatenate all DataFrames
big_error_df  = pd.concat(df_list, ignore_index=True)

# Cria Tb Aplicações
tb_medidas_error = big_error_df[[8, 6, 7, 5]].copy() # Colunas hash, hora, minuto, qtd de erros

# salva arquivo
tb_medidas_error.to_csv(f'{path_projeto}/{path_processados}/consolidado_error.csv', index=False, header=['hash', 'hora', 'min', 'qtd']) 
    
print('Gerado arquivo consolidado da métrica de erros de Memória')

Gerado arquivo consolidado da métrica de erros de Memória


In [10]:
# Recuperar metricas dos arquivos processados
path_processados = '/dados/coletas_dia_3/throttled/processados'
csv_files = glob.glob(path_projeto+path_processados + "/*.csv")

df_list = (pd.read_csv(file, skiprows = 1,header = None) for file in csv_files)

# Concatenate all DataFrames
big_throttled_df  = pd.concat(df_list, ignore_index=True)

# Cria Tb Aplicações
tb_medidas_throttled = big_throttled_df[[7, 5, 6, 4]].copy() # Colunas hash, hora, minuto, qtd de erros

# salva arquivo
tb_medidas_throttled.to_csv(f'{path_projeto}/{path_processados}/consolidado_cpu_throttled.csv', index=False, header=['hash', 'hora', 'min', 'consumo_cpu']) 
    
print('Gerado arquivo consolidado da métrica de consumo excessivo de cpu')

Gerado arquivo consolidado da métrica de consumo excessivo de cpu


## Agregação dos Dados e Limpeza

In [11]:
## Agregação dos valores de CPU
path_processados_cpu = '/dados/coletas_dia_3/cpu/processados'
csv_file_consolidado_cpu = f'{path_projeto}{path_processados_cpu}/consolidado_cpu.csv'
df_consolidado_cpu = pd.read_csv(csv_file_consolidado_cpu, sep=',',decimal='.')
# Remoção dos valores zerados de consumo de CPU
df_consolidado_cpu = df_consolidado_cpu[df_consolidado_cpu['consumo_cpu'] > 0]
# Transformação do consumo de CPU em miliCPU (miliCPU) (x 1000)
df_consolidado_cpu['consumo_cpu'] = df_consolidado_cpu['consumo_cpu'] * 1000
# Exclui as colunas de hora e minuto
del df_consolidado_cpu['hora']
del df_consolidado_cpu['min']
# Realiza agrupamento do consumo de CPU por aplicação, identififcado pela coluna hash. 
# Será considerado os valores de pico (max) por aplicação
df_group_by_max_cpu = df_consolidado_cpu.groupby(['hash']).max()
# Reseta o índice
df_group_by_max_cpu.reset_index(inplace=True)
df_group_by_max_cpu = df_group_by_max_cpu.rename(columns = {'consumo_cpu':'pico_cpu'})
# Agrupa as aplicações e agrega pelo valor da mediana dos valores de consumo de CPU por aplicação
df_group_by_med_cpu = df_consolidado_cpu.groupby(['hash']).median()
# Reseta o índice
df_group_by_med_cpu.reset_index(inplace=True)
df_group_by_med_cpu = df_group_by_med_cpu.rename(columns = {'consumo_cpu':'mediana_cpu'})
print('Gerados valores de CPU: valor de pico e mediana')

Gerados valores de CPU: valor de pico e mediana


In [12]:
## Agregação dos dados de Memória
path_processados_memoria = '/dados/coletas_dia_3/memoria/processados'
csv_file_consolidado_memoria = f'{path_projeto}{path_processados_memoria}/consolidado_memoria.csv'
df_consolidado_memoria = pd.read_csv(csv_file_consolidado_memoria, sep=',',decimal='.')
# Remove os valores zerados de consumo de memória
df_consolidado_memoria = df_consolidado_memoria[df_consolidado_memoria['consumo_memoria'] > 0]
# Exclui a coluna de hora e minuto
del df_consolidado_memoria['hora']
del df_consolidado_memoria['min']
## Connverte o valor de consumo de memória de bytes para MB
df_consolidado_memoria['consumo_memoria'] = df_consolidado_memoria['consumo_memoria'] / (1024 * 1024)
# Obtenção do Pico de Memória por aplicação
df_group_by_max_memoria = df_consolidado_memoria.groupby(['hash']).max()
# Reseta o índice
df_group_by_max_memoria.reset_index(inplace=True)
df_group_by_max_memoria = df_group_by_max_memoria.rename(columns = {'consumo_memoria':'pico_memoria'})
# Obtenção da Mediana de Consumo de Memória
df_group_by_med_memoria = df_consolidado_memoria.groupby(['hash']).median()
# Reseta o índice
df_group_by_med_memoria.reset_index(inplace=True)
df_group_by_med_memoria = df_group_by_med_memoria.rename(columns = {'consumo_memoria':'mediana_memoria'})
print('Gerados valores de Memoria: valor de pico e mediana')

Gerados valores de Memoria: valor de pico e mediana


### Agregação das métricas num único dataframe

In [13]:
# Realizando o join do consumo de memória e cpu
# Agregação das colunas de valores mediana e pico da CPU
df_group_by_cpu = pd.concat([df_group_by_med_cpu.set_index('hash'),df_group_by_max_cpu.set_index('hash')], 
                            axis=1, join='inner').reset_index()

# Agregação das colunas de valores mediana e pico da Memoria
df_group_by_memoria = pd.concat([df_group_by_med_memoria.set_index('hash'),df_group_by_max_memoria.set_index('hash')], 
                                axis=1, join='inner').reset_index()

# Agregação das colunas de CPU e Memoria
df_group_by_cpu_memoria = pd.concat([df_group_by_cpu.set_index('hash'),df_group_by_memoria.set_index('hash')], 
                                    axis=1, join='inner').reset_index()
df_group_by_cpu_memoria

,hash,mediana_cpu,pico_cpu,mediana_memoria,pico_memoria
0,00115d136a63167e961d60147809aff7,129.762665,255.885808,208.097656,208.460938
1,00171f1ecb4cb384c73b460e424033ae,10.674630,17.904220,129.964844,138.015625
2,00190e6b7ce5fa82d9e75b96948f244e,3.239884,3.807161,127.517578,130.609375
3,00192f6f6f4e5a93d8d562202c6d0227,0.856028,0.939062,485.562500,485.734375
4,001d79ecf18bc2bb98c8ac7dc5ab4526,2.546197,4.612576,127.632812,131.617188
...,...,...,...,...,...
13352,ffd950eb6ceeacfbd2f239eb34679cbe,1.043283,3.513072,674.507812,674.777344
13353,ffdb935e97a6dd0530958526e31dc129,3.342989,7.209065,732.753906,1022.218750
13354,ffe7450b3571c80c26389cd403b6017c,3.338460,7.572723,180.544922,249.750000
13355,fff13383e3a6e8b8cc87a597f9fc8272,4.123843,61.843285,3197.683594,3253.917969


### Remoção da aplicações com execuções além dos recursos de cpu e memória

In [14]:
## Agrupa as aplicacoes com estouro de memória
path_processados_error = '/dados/coletas_dia_3/error/processados'
csv_file_consolidado_error = f'{path_projeto}{path_processados_error}/consolidado_error.csv'
df_consolidado_error_memoria = pd.read_csv(csv_file_consolidado_error, sep=',',decimal='.')
## Agrega as aplicações pelo o hash de identificação
df_aplicacoes_error_memoria = df_consolidado_error_memoria['hash'].unique()
## Remove as aplicaçoes com erro de memória do dataframe consolidado
df_group_by_cpu_memoria = df_group_by_cpu_memoria[~df_group_by_cpu_memoria['hash'].isin(df_aplicacoes_error_memoria)]
## Agrupa as aplicacoes com excesso de cpu (throttled)
path_processados_throttled = '/dados/coletas_dia_3/throttled/processados'
csv_file_consolidado_throttled = f'{path_projeto}{path_processados_throttled}/consolidado_cpu_throttled.csv'
df_consolidado_throttled = pd.read_csv(csv_file_consolidado_throttled, sep=',',decimal='.')
## Agrega as aplicações pelo o hash de identificação
df_consolidado_throttled = df_consolidado_throttled['hash'].unique()
## Remove as aplicaçoes com erro de memória do dataframe consolidado
df_group_by_cpu_memoria = df_group_by_cpu_memoria[~df_group_by_cpu_memoria['hash'].isin(df_consolidado_throttled)]
df_group_by_cpu_memoria.reset_index(drop=True)
print('Removido aplicacoes (conteineres) com execuções além dos recursos de cpu e memoria')

Removido aplicacoes (conteineres) com execuções além dos recursos de cpu e memoria


In [15]:
### Remoção de valores discrepantes (acima do flavor f10)
df_group_by_cpu_memoria = df_group_by_cpu_memoria[df_group_by_cpu_memoria['pico_cpu'] < 10.8]
df_group_by_cpu_memoria = df_group_by_cpu_memoria[df_group_by_cpu_memoria['pico_memoria'] < 550]

### Rotulação

In [16]:
## Flavors
flavors = [{'id': '0', 'flavor': 'f0', 'cpu': 1, 'memoria': 50},
           {'id': '1', 'flavor': 'f1', 'cpu': 2, 'memoria': 100},
           {'id': '2', 'flavor': 'f2', 'cpu': 3, 'memoria': 150},
           {'id': '3', 'flavor': 'f3', 'cpu': 4, 'memoria': 200},
           {'id': '4', 'flavor': 'f4', 'cpu': 5, 'memoria': 250},
           {'id': '5', 'flavor': 'f5', 'cpu': 6, 'memoria': 300},
           {'id': '6', 'flavor': 'f6', 'cpu': 7, 'memoria': 350},
           {'id': '7', 'flavor': 'f7', 'cpu': 8, 'memoria': 400},
           {'id': '8', 'flavor': 'f8', 'cpu': 9, 'memoria': 450},
           {'id': '9', 'flavor': 'f9', 'cpu': 10, 'memoria': 500},
           {'id': '10', 'flavor': 'f10', 'cpu': 11, 'memoria': 550}]

## Thresholds
cpu_threshod = 0.05
memoria_threshod = 0.1

## Atribuição do flavor
df_group_by_cpu_memoria['flavor'] = df_group_by_cpu_memoria.apply(lambda x: rotular_V2(x.pico_cpu, x.mediana_cpu, cpu_threshod, x.pico_memoria, x.mediana_memoria, memoria_threshod), axis=1)
print ('Rotulacao Completa!')

Rotulacao Completa!


In [17]:
print('Quantidade de conteineres não classificados ', len(df_group_by_cpu_memoria[df_group_by_cpu_memoria['flavor']=='Nao classificado']))

Quantidade de conteineres não classificados  2


In [18]:
## Remove os não classificados
## Os não classificados deveriam estar num próximo flavor f11, caso existisse. Neste caso serão removidos.
df_nao_classificado = df_group_by_cpu_memoria[df_group_by_cpu_memoria['flavor']=='Nao classificado']
df_nao_classificado

,hash,mediana_cpu,pico_cpu,mediana_memoria,pico_memoria,flavor
4481,546937e47847d74a02674f17a74536e8,1.431043,2.274491,549.464844,549.925781,Nao classificado
6567,7bf6caf2f2abdc66445c0a231cd580dd,3.974121,5.606177,548.808594,549.972656,Nao classificado


### Classificação dos Dados com o Modelo já treinado

In [19]:
## Carrega o modelo
# Loading the saved decision tree model pickle
import pickle
decision_tree_pkl_filename = 'decision_tree_classifier_entropy.pkl'
decision_tree_model_pkl = open(decision_tree_pkl_filename, 'rb')
decision_tree_model = pickle.load(decision_tree_model_pkl)
print ("Carregado o modelo : ", decision_tree_model)

Carregado o modelo :  DecisionTreeClassifier(criterion='entropy')


### Predições e Validação com o novo dataset

In [20]:
df_group_by_cpu_memoria.head()

,hash,mediana_cpu,pico_cpu,mediana_memoria,pico_memoria,flavor
2,00190e6b7ce5fa82d9e75b96948f244e,3.239884,3.807161,127.517578,130.609375,f3
3,00192f6f6f4e5a93d8d562202c6d0227,0.856028,0.939062,485.562500,485.734375,f9
4,001d79ecf18bc2bb98c8ac7dc5ab4526,2.546197,4.612576,127.632812,131.617188,f4
9,00397e755d86f88d58f23419ce8ba8e8,0.001729,0.002905,11.078125,11.082031,f0
10,004156954736d58869a3a2e1d5e197e8,6.623067,7.728354,353.800781,354.683594,f7


In [21]:
# Teste com escolha manual
exemplar = df_group_by_cpu_memoria[df_group_by_cpu_memoria['hash']=='00192f6f6f4e5a93d8d562202c6d0227']
predict(decision_tree_model, exemplar)

Predicao para os parâmetros [   mediana_cpu  pico_cpu  mediana_memoria  pico_memoria
3     0.856028  0.939062         485.5625    485.734375] => flavor f9


In [22]:
# Teste com escolha manual
exemplar = df_group_by_cpu_memoria[df_group_by_cpu_memoria['hash']=='00397e755d86f88d58f23419ce8ba8e8']
predict(decision_tree_model, exemplar)

Predicao para os parâmetros [   mediana_cpu  pico_cpu  mediana_memoria  pico_memoria
9     0.001729  0.002905        11.078125     11.082031] => flavor f0


In [23]:
## Remove não classificados
for i in df_nao_classificado.index:
    df_group_by_cpu_memoria = df_group_by_cpu_memoria.drop([i])
print('Quantidade de conteineres não classificados ', len(df_group_by_cpu_memoria[df_group_by_cpu_memoria['flavor']=='Nao classificado']))

Quantidade de conteineres não classificados  0


In [25]:
X_test = df_group_by_cpu_memoria[['mediana_cpu', 'pico_cpu', 'mediana_memoria', 'pico_memoria']]
y_test = df_group_by_cpu_memoria[['flavor']]
allScores = cross_val_score(decision_tree_model, X_test, y_test , cv=10)
# cross_val_score retorna array com as 10 validações
allScores.mean() # tomamos a média do score

0.9792554737995662

In [26]:
# Prediction
y_pred = decision_tree_model.predict(X_test)
cal_accuracy(y_test, y_pred)

Confusion Matrix:  [[1703    0    0    0    0    0    0    0    0    0    0]
 [   2 1172    0    6    0    0    0    0    0    0    0]
 [   0    0  277    0    0    0    0    0    0    0   12]
 [   0    6    0  763    6    0    0    0    0    0    0]
 [   0    0    0    8  531   10    0    0    0    0    0]
 [   0    0    0    0    5  489    5    0    0    0    0]
 [   0    0    0    0    0    5  449   10    0    0    0]
 [   0    0    0    0    0    0    5  452    9    0    0]
 [   0    0    0    0    0    0    0    7  517   13    0]
 [   0    0    0    0    0    0    0    0   10  441   12]
 [   0    0   18    0    0    0    0    0    1   16  366]]
Accuracy :  97.73409773409773
Report :                precision    recall  f1-score   support

          f0       1.00      1.00      1.00      1703
          f1       0.99      0.99      0.99      1180
         f10       0.94      0.96      0.95       289
          f2       0.98      0.98      0.98       775
          f3       0.98      0.